# Detecting spouse mentions in sentences

In this tutorial, we will see how Snorkel can be used for Information Extraction. We will walk through an example text classification task for information extraction, where we use labeling functions involving keywords and distant supervision.
### Classification Task
<img src="imgs/sentence.jpg" width="700px;" onerror="this.onerror=null; this.src='/doks-theme/assets/images/sentence.jpg';" align="center" style="display: block; margin-left: auto; margin-right: auto;">

We want to classify each __candidate__ or pair of people mentioned in a sentence, as being married at some point or not.

In the above example, our candidate represents the possible relation `(Barack Obama, Michelle Obama)`. As readers, we know this mention is true due to external knowledge and the keyword of `wedding` occuring later in the sentence.
We begin with some basic setup and data downloading.


In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import pickle
import numpy as np

if os.path.basename(os.getcwd()) == "snorkel-tutorials":
    os.chdir("spouse")

In [ ]:
from utils import load_data

((df_dev, Y_dev), df_train, (df_test, Y_test)) = load_data()
print(df_dev.shape, df_train.shape, df_test.shape)

**Input Data:** `df_dev`, `df_train`, and `df_test` are `Pandas DataFrame` objects, where each row represents a particular __candidate__. For our problem, a candidate consists of a sentence, and two people mentioned in the sentence. The DataFrames contain the fields `sentence`, which refers to the sentence of the candidate, `tokens`, the tokenized form of the sentence, and `person1_word_idx` and `person2_word_idx`, which represent `[start, end]` indices in the tokens at which the first and second person's name appear, respectively.

We also have certain **preprocessed fields**, that we discuss a few cells below.

In [ ]:

# Don't truncate text fields in the display
pd.set_option("display.max_colwidth", 0)

df_dev.head()

Let's look at a candidate in the development set:

In [ ]:
from preprocessors import get_person_text

candidate = df_dev.loc[2]
person_names = get_person_text(candidate).person_names

print("Sentence: ", candidate["sentence"])
print("Person 1: ", person_names[0])
print("Person 2: ", person_names[1])

### Preprocessing the Data

In a real application, there is a lot of data preparation, parsing, and database loading that needs to be completed before we generate candidates and dive into writing labeling functions. Here we've pre-generated candidates in a pandas DataFrame object per split (train,dev,test).

### Labeling Function Helpers

When writing labeling functions, there are several functions you will use over and over again. In the case of text relation extraction as with this task, common functions include those for fetching text between mentions of the two people in a candidate, examing word windows around person mentions, and so on. We will wrap these functions as `preprocessors`.

In [ ]:
from snorkel.preprocess import preprocessor


@preprocessor()
def get_text_between(cand):
    """
    Returns the text between the two person mentions in the sentence for a candidate
    """
    start = cand.person1_word_idx[1] + 1
    end = cand.person2_word_idx[0]
    cand.text_between = " ".join(cand.tokens[start:end])
    return cand

### Candidate PreProcessors

For the purposes of the tutorial, we have three fields (`between_tokens`, `person1_right_tokens`, `person2_right_tokens`) preprocessed in the data, which can be used when creating labeling functions. We also provide the following set of `preprocessor`s for this task in `preprocessors.py`, along with the fields these populate.
* `get_person_text(cand)`: `person_names`
* `get_person_lastnames(cand)`: `person_lastnames`
* `get_left_tokens(cand)`: `person1_left_tokens`, `person2_left_tokens`

In [ ]:
from preprocessors import get_left_tokens, get_person_last_names

POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1

In [ ]:
from snorkel.labeling import labeling_function

# Check for the `spouse` words appearing between the person mentions
spouses = {"spouse", "wife", "husband", "ex-wife", "ex-husband"}


@labeling_function(resources=dict(spouses=spouses))
def lf_husband_wife(x, spouses):
    return POSITIVE if len(spouses.intersection(set(x.between_tokens))) > 0 else ABSTAIN

In [ ]:
# Check for the `spouse` words appearing to the left of the person mentions
@labeling_function(resources=dict(spouses=spouses), pre=[get_left_tokens])
def lf_husband_wife_left_window(x, spouses):
    if len(set(spouses).intersection(set(x.person1_left_tokens))) > 0:
        return POSITIVE
    elif len(set(spouses).intersection(set(x.person2_left_tokens))) > 0:
        return POSITIVE
    else:
        return ABSTAIN

In [ ]:
# Check for the person mentions having the same last name
@labeling_function(pre=[get_person_last_names])
def lf_same_last_name(x):
    p1_ln, p2_ln = x.person_lastnames

    if p1_ln and p2_ln and p1_ln == p2_ln:
        return POSITIVE
    return ABSTAIN

In [ ]:
# Check for the word `married` between person mentions
@labeling_function()
def lf_married(x):
    return POSITIVE if "married" in x.between_tokens else ABSTAIN

In [ ]:
# Check for words that refer to `family` relationships between and to the left of the person mentions
family = {
    "father",
    "mother",
    "sister",
    "brother",
    "son",
    "daughter",
    "grandfather",
    "grandmother",
    "uncle",
    "aunt",
    "cousin",
}
family = family.union({f + "-in-law" for f in family})


@labeling_function(resources=dict(family=family))
def lf_familial_relationship(x, family):
    return NEGATIVE if len(family.intersection(set(x.between_tokens))) > 0 else ABSTAIN


@labeling_function(resources=dict(family=family), pre=[get_left_tokens])
def lf_family_left_window(x, family):
    if len(set(family).intersection(set(x.person1_left_tokens))) > 0:
        return NEGATIVE
    elif len(set(family).intersection(set(x.person2_left_tokens))) > 0:
        return NEGATIVE
    else:
        return ABSTAIN

In [ ]:
# Check for `other` relationship words between person mentions
other = {"boyfriend", "girlfriend", "boss", "employee", "secretary", "co-worker"}


@labeling_function(resources=dict(other=other))
def lf_other_relationship(x, other):
    return NEGATIVE if len(other.intersection(set(x.between_tokens))) > 0 else ABSTAIN

### Distant Supervision Labeling Functions

In addition to using factories that encode pattern matching heuristics, we can also write labeling functions that _distantly supervise_ data points. Here, we'll load in a list of known spouse pairs and check to see if the pair of persons in a candidate matches one of these.

[**DBpedia**](http://wiki.dbpedia.org/): Our database of known spouses comes from DBpedia, which is a community-driven resource similar to Wikipedia but for curating structured data. We'll use a preprocessed snapshot as our knowledge base for all labeling function development.

We can look at some of the example entries from DBPedia and use them in a simple distant supervision labeling function.

Make sure `dbpedia.pkl` is in the `spouse/data` directory.

In [ ]:
with open("data/dbpedia.pkl", "rb") as f:
    known_spouses = pickle.load(f)

list(known_spouses)[0:5]

In [ ]:
@labeling_function(resources=dict(known_spouses=known_spouses), pre=[get_person_text])
def lf_distant_supervision(x, known_spouses):
    p1, p2 = x.person_names
    if (p1, p2) in known_spouses or (p2, p1) in known_spouses:
        return POSITIVE
    else:
        return ABSTAIN

In [ ]:
from preprocessors import last_name

# Last name pairs for known spouses
last_names = set(
    [
        (last_name(x), last_name(y))
        for x, y in known_spouses
        if last_name(x) and last_name(y)
    ]
)


@labeling_function(resources=dict(last_names=last_names), pre=[get_person_last_names])
def lf_distant_supervision_last_names(x, last_names):
    p1_ln, p2_ln = x.person_lastnames

    return (
        POSITIVE
        if (p1_ln != p2_ln)
        and ((p1_ln, p2_ln) in last_names or (p2_ln, p1_ln) in last_names)
        else ABSTAIN
    )

#### Apply Labeling Functions to the Data
We create a list of labeling functions and apply them to the data

In [ ]:
from snorkel.labeling import PandasLFApplier

lfs = [
    lf_husband_wife,
    lf_husband_wife_left_window,
    lf_same_last_name,
    lf_married,
    lf_familial_relationship,
    lf_family_left_window,
    lf_other_relationship,
    lf_distant_supervision,
    lf_distant_supervision_last_names,
]
applier = PandasLFApplier(lfs)

In [ ]:
from snorkel.labeling import LFAnalysis

L_dev = applier.apply(df_dev)
L_train = applier.apply(df_train)

In [ ]:
LFAnalysis(L_dev, lfs).lf_summary(Y_dev)

## Imports for dependencies and informed snorkel label_model

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
from Our_Monitors.CD_Monitor import CDM, Informed_LabelModel
from Our_Monitors.CDGA_Monitor import CDGAM
from Our_Monitors.New_Monitor import NM
from Our_Monitors.utils import ModVarma_InCov

from dependency_model.varma_deps_functions import get_varma_edges, get_varma_with_gold_edges

## Validation Analysis

#### Toggle all warnings js code

In [ ]:
%%javascript
(function(on) {
const e=$( "<a>Setup failed</a>" );
const ns="js_jupyter_suppress_warnings";
var cssrules=$("#"+ns);
if(!cssrules.length) cssrules = $("<style id='"+ns+"' type='text/css'>div.output_stderr { } </style>").appendTo("head");
e.click(function() {
    var s='Showing';  
    cssrules.empty()
    if(on) {
        s='Hiding';
        cssrules.append("div.output_stderr, div[data-mime-type*='.stderr'] { display:none; }");
    }
    e.text(s+' warnings (click to toggle)');
    on=!on;
}).click();
$(element).append(e);
})(true);

In [ ]:
L_test = applier.apply(df_test)

In [ ]:
%%time
from snorkel.analysis import metric_score
from snorkel.utils import probs_to_preds

epochs_list = [50, 100, 500, 1000, 5000][::-1]
sig_list = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9]
sig_crazy_list = [0.75, 0.9]
thresh_list = [0.1, 0.5, 1, 1.5]
lr_list = [0.01, 0.025, 0.05, 0.1]

deps_names = ['NM', 'CDGAM', 'Varma_Gold', 'Varma', 'Empty'] # !!!!!!!!!! modify this !!!!!!!!!!!

info = { 'CDM': {'deps_params': sig_list, 'snorkel_eps': epochs_list, 'snorkel_lr':lr_list}, 
            'NM': {'deps_params': sig_list, 'snorkel_eps': epochs_list, 'snorkel_lr':lr_list}, 
            'NM_NP': {'deps_params': sig_list, 'snorkel_eps': epochs_list, 'snorkel_lr':lr_list}, 
            'CDGAM': {'deps_params': sig_list, 'snorkel_eps': epochs_list, 'snorkel_lr':lr_list}, 
            'Mod_Varma': {'deps_params': thresh_list, 'snorkel_eps': epochs_list, 'snorkel_lr':lr_list}, 
            'Varma': {'deps_params': thresh_list, 'snorkel_eps': epochs_list, 'snorkel_lr':lr_list}, 
            'Varma_Gold': {'deps_params': thresh_list, 'snorkel_eps': epochs_list, 'snorkel_lr':lr_list}, 
            'Empty': {'deps_params': [-1], 'snorkel_eps': epochs_list, 'snorkel_lr':lr_list}}

roc_based_store = {key: {'param': -1, 'n_eps': -1, 'lr': -1, 'f1': -1, 'roc_auc': -1} for key in deps_names}

f1_based_store = {key: {'param': -1, 'n_eps': -1, 'lr': -1, 'f1': -1, 'roc_auc': -1} for key in deps_names}

def overall_deps_fn(deps_name, param):
    if deps_name == 'CDM':
        deps = CDM(L_dev, Y_dev, k=2, sig=param, policy = 'old', verbose=False, return_more_info = False)
    elif deps_name == 'CDGAM':
        deps = CDGAM(L_dev, k=2, sig=param, policy = 'old', verbose = False, return_more_info = False)
    elif deps_name == 'NM':
        deps = NM(L_dev, Y_dev, k=2, sig=param, policy = 'old', verbose=False, return_more_info = False)
    elif deps_name == 'NM_NP':
        deps = NM(L_dev, Y_dev, k=2, sig=param, policy = 'new', verbose=False, return_more_info = False)
    elif deps_name == 'Mod_Varma':
        deps = ModVarma_InCov(L_dev, Y_dev, thresh=param)
    elif deps_name == 'Varma':
        deps = get_varma_edges(L_dev, thresh=param)
    elif deps_name == 'Varma_Gold':
        deps = get_varma_with_gold_edges(L_dev, Y_dev, thresh=param)
    elif deps_name == 'Empty':
        deps = []
    return deps

ct=0
total = sum([len(info[deps_name]['deps_params']) for deps_name in deps_names]) * len(epochs_list) * len(lr_list)
for deps_name in deps_names:
    for param in info[deps_name]['deps_params']:
        deps = overall_deps_fn(deps_name, param)
        
        for n_eps in info[deps_name]['snorkel_eps']:
            for lr in info[deps_name]['snorkel_lr']:
                
                label_model = Informed_LabelModel(edges = deps, cardinality=2, verbose=True)
                label_model.fit(L_train, Y_dev, seed=12345, lr=lr, log_freq=n_eps/10, n_epochs=n_eps)
                
                probs_dev = label_model.predict_proba(L_dev)
                preds_dev = probs_to_preds(probs_dev)
                f1 = metric_score(Y_dev, preds_dev, probs=probs_dev, metric='f1')
                roc_auc = metric_score(Y_dev, preds_dev, probs=probs_dev, metric='roc_auc')
                
                if roc_auc>roc_based_store[deps_name]['roc_auc']:
                    print(deps_name, param, deps, n_eps, lr, " | roc: ", roc_auc)
                    roc_based_store[deps_name] = {'param': param, 'n_eps': n_eps, 'lr': lr, 'f1': f1, 'roc_auc': roc_auc}
                
                if f1>f1_based_store[deps_name]['f1']:
                    print(deps_name, param, deps, n_eps, lr, " | f1: ", f1)
                    f1_based_store[deps_name] = {'param': param, 'n_eps': n_eps, 'lr': lr, 'f1': f1, 'roc_auc': roc_auc}   
                
                ct +=1
                print(ct, " / ", total)


In [ ]:
%%time
# test using params from validation
f1_based_test_store = {key: {'f1': -1, 'roc_auc': -1} for key in deps_names}
roc_based_test_store = {key: {'f1': -1, 'roc_auc': -1} for key in deps_names}

for deps_name in deps_names:
    param = f1_based_store[deps_name]['param']
    n_eps = f1_based_store[deps_name]['n_eps']
    lr = f1_based_store[deps_name]['lr']
    
    deps = overall_deps_fn(deps_name, param)
    
    label_model = Informed_LabelModel(edges = deps, cardinality=2, verbose=True)
    label_model.fit(L_train, Y_dev, seed=12345, lr=lr, log_freq=n_eps/10, n_epochs=n_eps)
    
    probs_test = label_model.predict_proba(L_test)
    preds_test = probs_to_preds(probs_test)
    f1_test = metric_score(Y_test, preds_test, probs=probs_test, metric='f1')
    roc_auc_test = metric_score(Y_test, preds_test, probs=probs_test, metric='roc_auc')

    f1_based_test_store[deps_name] = {'f1': f1_test, 'roc_auc': roc_auc_test}
    
    
for deps_name in deps_names:
    param = roc_based_store[deps_name]['param']
    n_eps = roc_based_store[deps_name]['n_eps']
    lr = roc_based_store[deps_name]['lr']
    
    deps = overall_deps_fn(deps_name, param)
    
    label_model = Informed_LabelModel(edges = deps, cardinality=2, verbose=True)
    label_model.fit(L_train, Y_dev, seed=12345, lr=lr, log_freq=n_eps/10, n_epochs=n_eps)
    
    probs_test = label_model.predict_proba(L_test)
    preds_test = probs_to_preds(probs_test)
    f1_test = metric_score(Y_test, preds_test, probs=probs_test, metric='f1')
    roc_auc_test = metric_score(Y_test, preds_test, probs=probs_test, metric='roc_auc')

    roc_based_test_store[deps_name] = {'f1': f1_test, 'roc_auc': roc_auc_test}




In [ ]:
print(pd.DataFrame(roc_based_store))
print(pd.DataFrame(roc_based_test_store))

In [ ]:
print(pd.DataFrame(f1_based_store))
print(pd.DataFrame(f1_based_test_store))

### Rerun above validation analysis for our methods without very large significance levels

In [ ]:
%%time
from snorkel.analysis import metric_score
from snorkel.utils import probs_to_preds

epochs_list = [50, 100, 500, 1000, 5000][::-1]
sig_list = [0.01, 0.05, 0.1, 0.25]
sig_crazy_list = [0.75, 0.9]
thresh_list = [0.1, 0.5, 1, 1.5]
lr_list = [0.01, 0.025, 0.05, 0.1]

deps_names = ['CDGAM'] # !!!!!!!!!! modify this !!!!!!!!!!!

info = { 'CDM': {'deps_params': sig_list, 'snorkel_eps': epochs_list, 'snorkel_lr':lr_list}, 
            'NM': {'deps_params': sig_list, 'snorkel_eps': epochs_list, 'snorkel_lr':lr_list}, 
            'NM_NP': {'deps_params': sig_list, 'snorkel_eps': epochs_list, 'snorkel_lr':lr_list}, 
            'CDGAM': {'deps_params': sig_list, 'snorkel_eps': epochs_list, 'snorkel_lr':lr_list}, 
            'Mod_Varma': {'deps_params': thresh_list, 'snorkel_eps': epochs_list, 'snorkel_lr':lr_list}, 
            'Varma': {'deps_params': thresh_list, 'snorkel_eps': epochs_list, 'snorkel_lr':lr_list}, 
            'Varma_Gold': {'deps_params': thresh_list, 'snorkel_eps': epochs_list, 'snorkel_lr':lr_list}, 
            'Empty': {'deps_params': [-1], 'snorkel_eps': epochs_list, 'snorkel_lr':lr_list}}

roc_based_store = {key: {'param': -1, 'n_eps': -1, 'lr': -1, 'f1': -1, 'roc_auc': -1} for key in deps_names}

f1_based_store = {key: {'param': -1, 'n_eps': -1, 'lr': -1, 'f1': -1, 'roc_auc': -1} for key in deps_names}

def overall_deps_fn(deps_name, param):
    if deps_name == 'CDM':
        deps = CDM(L_dev, Y_dev, k=2, sig=param, policy = 'old', verbose=False, return_more_info = False)
    elif deps_name == 'CDGAM':
        deps = CDGAM(L_dev, k=2, sig=param, policy = 'old', verbose = False, return_more_info = False)
    elif deps_name == 'NM':
        deps = NM(L_dev, Y_dev, k=2, sig=param, policy = 'old', verbose=False, return_more_info = False)
    elif deps_name == 'NM_NP':
        deps = NM(L_dev, Y_dev, k=2, sig=param, policy = 'new', verbose=False, return_more_info = False)
    elif deps_name == 'Mod_Varma':
        deps = ModVarma_InCov(L_dev, Y_dev, thresh=param)
    elif deps_name == 'Varma':
        deps = get_varma_edges(L_dev, thresh=param)
    elif deps_name == 'Varma_Gold':
        deps = get_varma_with_gold_edges(L_dev, Y_dev, thresh=param)
    elif deps_name == 'Empty':
        deps = []
    return deps

ct=0
total = sum([len(info[deps_name]['deps_params']) for deps_name in deps_names]) * len(epochs_list) * len(lr_list)
for deps_name in deps_names:
    for param in info[deps_name]['deps_params']:
        deps = overall_deps_fn(deps_name, param)
        
        for n_eps in info[deps_name]['snorkel_eps']:
            for lr in info[deps_name]['snorkel_lr']:
                
                label_model = Informed_LabelModel(edges = deps, cardinality=2, verbose=True)
                label_model.fit(L_train, Y_dev, seed=12345, lr=lr, log_freq=n_eps/10, n_epochs=n_eps)
                
                probs_dev = label_model.predict_proba(L_dev)
                preds_dev = probs_to_preds(probs_dev)
                f1 = metric_score(Y_dev, preds_dev, probs=probs_dev, metric='f1')
                roc_auc = metric_score(Y_dev, preds_dev, probs=probs_dev, metric='roc_auc')
                
                if roc_auc>roc_based_store[deps_name]['roc_auc']:
                    print(deps_name, param, deps, n_eps, lr, " | roc: ", roc_auc)
                    roc_based_store[deps_name] = {'param': param, 'n_eps': n_eps, 'lr': lr, 'f1': f1, 'roc_auc': roc_auc}
                
                if f1>f1_based_store[deps_name]['f1']:
                    print(deps_name, param, deps, n_eps, lr, " | f1: ", f1)
                    f1_based_store[deps_name] = {'param': param, 'n_eps': n_eps, 'lr': lr, 'f1': f1, 'roc_auc': roc_auc}   
                
                ct +=1
                print(ct, " / ", total)


In [ ]:
%%time
# test using params from validation
f1_based_test_store = {key: {'f1': -1, 'roc_auc': -1} for key in deps_names}
roc_based_test_store = {key: {'f1': -1, 'roc_auc': -1} for key in deps_names}

for deps_name in deps_names:
    param = f1_based_store[deps_name]['param']
    n_eps = f1_based_store[deps_name]['n_eps']
    lr = f1_based_store[deps_name]['lr']
    
    deps = overall_deps_fn(deps_name, param)
    
    label_model = Informed_LabelModel(edges = deps, cardinality=2, verbose=True)
    label_model.fit(L_train, Y_dev, seed=12345, lr=lr, log_freq=n_eps/10, n_epochs=n_eps)
    
    probs_test = label_model.predict_proba(L_test)
    preds_test = probs_to_preds(probs_test)
    f1_test = metric_score(Y_test, preds_test, probs=probs_test, metric='f1')
    roc_auc_test = metric_score(Y_test, preds_test, probs=probs_test, metric='roc_auc')

    f1_based_test_store[deps_name] = {'f1': f1_test, 'roc_auc': roc_auc_test}
    
    
for deps_name in deps_names:
    param = roc_based_store[deps_name]['param']
    n_eps = roc_based_store[deps_name]['n_eps']
    lr = roc_based_store[deps_name]['lr']
    
    deps = overall_deps_fn(deps_name, param)
    
    label_model = Informed_LabelModel(edges = deps, cardinality=2, verbose=True)
    label_model.fit(L_train, Y_dev, seed=12345, lr=lr, log_freq=n_eps/10, n_epochs=n_eps)
    
    probs_test = label_model.predict_proba(L_test)
    preds_test = probs_to_preds(probs_test)
    f1_test = metric_score(Y_test, preds_test, probs=probs_test, metric='f1')
    roc_auc_test = metric_score(Y_test, preds_test, probs=probs_test, metric='roc_auc')

    roc_based_test_store[deps_name] = {'f1': f1_test, 'roc_auc': roc_auc_test}




In [ ]:
print(pd.DataFrame(roc_based_store))
print(pd.DataFrame(roc_based_test_store))

In [ ]:
print(pd.DataFrame(f1_based_store))
print(pd.DataFrame(f1_based_test_store))